In [1]:
import os

os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/ra_joint_predictions')

In [2]:
from utils.config import Config

config = Config()

In [4]:
from dream.prediction import predict_dream_test_set

predict_params = {
        'hands_joint_source': './data/predictions/hand_joint_data_test.csv',
        'feet_joint_source': './data/predictions/feet_joint_data_test.csv',
        'hands_narrowing_model': { 'model': '../trained_models/adam_no_weights_reg/hands_narrowing_adam_no_weights_reg_shuffle.h5', 'is_regression': True },
        'wrists_narrowing_model': { 'model': '../trained_models/adam_no_weights_reg/wrists_narrowing_adam_no_weights_reg_shuffle.h5', 'is_regression': True },
        'feet_narrowing_model': { 'model': '../trained_models/adam_no_weights_reg/feet_narrowing_adam_no_weights_reg_shuffle_hand_pretrain.h5', 'is_regression': True },
        'hands_erosion_model': { 'model': '../trained_models/adam_no_weights_reg/hands_erosion_adam_no_weights_reg_shuffle.h5', 'is_regression': True },
        'wrists_erosion_model': { 'model': '../trained_models/adam_no_weights_reg/wrists_erosion_adam_no_weights_reg_shuffle.h5', 'is_regression': True },
        'feet_erosion_model': { 'model': '../trained_models/adam_no_weights_reg/feet_erosion_adam_no_weights_reg_shuffle_hand_pretrain.h5', 'is_regression': True },
        'template_path': '../resources/template.csv',
        'output_path': '../trained_models/adam_no_weights_reg_shuffle_hand_pretrain_val_output.csv',
    }

predict_dream_test_set(config, predict_params)